<a href="https://colab.research.google.com/github/sonleh96/wb-gpbp-ldt/blob/dev-ghinwa/Accessibility_AggregatingGID2_District.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**bold text**

In [ ]:
!pip install google-cloud-storage
import pandas as pd
import geopandas as gpd
import numpy as np

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.cloud import storage

# Create a client
client = storage.Client()

#Access the Google Bucket
bucket_name = 'wb-ldt'
bucket = client.get_bucket(bucket_name)

In [ ]:
#Name of Shape File
file_path = "RS/processed-data/SRB_access_tohealthcarefacilities.csv"
gcs_file_path = 'gs://' + bucket_name + '/' + file_path

#Read Shape File --> The shape file gives a MultiPolygon Geometry Column
df = pd.read_csv(gcs_file_path)

In [ ]:
df.head()

,GID_2,NAME_2,population_with_access,population_total,hospital_accesibility
0,SRB.1.1_1,Bor,27642.516381,40536.546387,68.0
1,SRB.1.2_1,Kladovo,NaN,17511.680623,0.0
2,SRB.1.3_1,Majdanpek,4627.089182,14174.625507,33.0
3,SRB.1.4_1,Negotin,2945.043685,30521.981085,10.0
4,SRB.10.1_1,Aleksinac,25760.257401,44732.588882,58.0


In [ ]:
#Name of Shape File
file_path = "shapefiles/gadm41_SRB_2.json"
gcs_file_path = 'gs://' + bucket_name + '/' + file_path

#Read Shape File --> The shape file gives a MultiPolygon Geometry Column
gdf = gpd.read_file(gcs_file_path)

#Adjust for GeoSpatial Data
center = gpd.GeoDataFrame(gdf[['GID_1', 'NAME_1']])

#Change the MultiPolygon Geometry Column to make it more useful
center['geometry'] = gdf.centroid
center = center.to_crs(gdf.crs)
center['lat'] = center.geometry.y
center['lon'] = center.geometry.x
#center

<ipython-input-6-bc2ae66525c0>:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center['geometry'] = gdf.centroid
<ipython-input-6-bc2ae66525c0>:12: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  center['geometry'] = gdf.centroid


In [ ]:
location_df = gdf[['GID_2', 'NAME_2', 'GID_1', 'NAME_1']]
location_df.head()

,GID_2,NAME_2,GID_1,NAME_1
0,SRB.1.1_1,Bor,SRB.1_1,Borski
1,SRB.1.2_1,Kladovo,SRB.1_1,Borski
2,SRB.1.3_1,Majdanpek,SRB.1_1,Borski
3,SRB.1.4_1,Negotin,SRB.1_1,Borski
4,SRB.2.1_1,Golubac,SRB.2_1,Braničevski


In [ ]:
df = df.merge(location_df, how = "outer", on = ["GID_2", "NAME_2"])
df.head()

,GID_2,NAME_2,population_with_access,population_total,hospital_accesibility,GID_1,NAME_1
0,SRB.1.1_1,Bor,27642.516381,40536.546387,68.0,SRB.1_1,Borski
1,SRB.1.2_1,Kladovo,NaN,17511.680623,0.0,SRB.1_1,Borski
2,SRB.1.3_1,Majdanpek,4627.089182,14174.625507,33.0,SRB.1_1,Borski
3,SRB.1.4_1,Negotin,2945.043685,30521.981085,10.0,SRB.1_1,Borski
4,SRB.10.1_1,Aleksinac,25760.257401,44732.588882,58.0,SRB.10_1,Nišavski


In [ ]:
df["population_with_access"] = df["population_with_access"].fillna(0)

In [ ]:
df_grouped = df.groupby(['GID_1', 'NAME_1']).agg(
    {'population_with_access': 'sum',
     'population_total':'sum'
     }
)

In [ ]:
df_grouped = df_grouped.reset_index()
df_grouped['hospital_accessibility_percentage'] = round(df_grouped['population_with_access']/df_grouped['population_total']*100, 2)

In [ ]:
df_grouped.head()

,GID_1,NAME_1,population_with_access,population_total,hospital_accessibility_percentage
0,SRB.10_1,Nišavski,225805.939194,361526.375984,62.46
1,SRB.11_1,Pčinjski,13331.431809,121099.224926,11.01
2,SRB.12_1,Pirotski,0.000000,77130.908744,0.00
3,SRB.13_1,Podunavski,3080.063083,170195.757029,1.81
4,SRB.14_1,Pomoravski,26003.791748,193731.237071,13.42


In [ ]:
df_grouped['year'] = 2022

In [ ]:
#Save the HealthCare Facilities in Serbia CSV file
from io import BytesIO

# Convert DataFrame to CSV in memory
csv_buffer = BytesIO()
df_grouped.to_csv(csv_buffer, index=False)

# Move the buffer's position to the beginning
csv_buffer.seek(0)

def upload_csv_to_gcs(bucket_name, destination_blob_name, file_buffer):
    """Uploads a file from a buffer to Google Cloud Storage."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    # Upload the file-like object (buffer)
    blob.upload_from_file(file_buffer, content_type='text/csv')
    print(f"File uploaded to {destination_blob_name}.")

# Define your GCS bucket name and destination path
destination_blob_name = 'RS/processed-data-district/SRB_access_tohealthcarefacilities_disctrict.csv'

# Upload the CSV from the buffer directly
upload_csv_to_gcs(bucket_name, destination_blob_name, csv_buffer)

File uploaded to RS/processed-data-district/SRB_access_tohealthcarefacilities_disctrict.csv.


In [ ]:
#Name of Shape File
file_path = "RS/processed-data/SRB_access_toschool.csv"
gcs_file_path = 'gs://' + bucket_name + '/' + file_path

#Read Shape File --> The shape file gives a MultiPolygon Geometry Column
df = pd.read_csv(gcs_file_path)

In [ ]:
df = df.merge(location_df, how = "outer", on = ["GID_2", "NAME_2"])
df.head()

,GID_2,NAME_2,population_with_access,population_total,school_accesibility,year,GID_1,NAME_1
0,SRB.1.1_1,Bor,27243.418347,40536.546387,67.0,2022,SRB.1_1,Borski
1,SRB.1.2_1,Kladovo,NaN,17511.680623,0.0,2022,SRB.1_1,Borski
2,SRB.1.3_1,Majdanpek,1693.121025,14174.625507,12.0,2022,SRB.1_1,Borski
3,SRB.1.4_1,Negotin,940.184842,30521.981085,3.0,2022,SRB.1_1,Borski
4,SRB.10.1_1,Aleksinac,19678.303280,44732.588882,44.0,2022,SRB.10_1,Nišavski


In [ ]:
df["population_with_access"] = df["population_with_access"].fillna(0)
df_grouped = df.groupby(['GID_1', 'NAME_1']).agg(
    {'population_with_access': 'sum',
     'population_total':'sum'
     }
)

In [ ]:
df_grouped = df_grouped.reset_index()
df_grouped['school_accessibility_percentage'] = round(df_grouped['population_with_access']/df_grouped['population_total']*100, 2)
df_grouped['year'] = 2022
df_grouped.head()

,GID_1,NAME_1,population_with_access,population_total,school_accessibility_percentage,year
0,SRB.10_1,Nišavski,229253.750444,361526.375984,63.41,2022
1,SRB.11_1,Pčinjski,41793.078464,121099.224926,34.51,2022
2,SRB.12_1,Pirotski,40152.638360,77130.908744,52.06,2022
3,SRB.13_1,Podunavski,44908.596730,170195.757029,26.39,2022
4,SRB.14_1,Pomoravski,65213.770466,193731.237071,33.66,2022


In [ ]:
#Save the HealthCare Facilities in Serbia CSV file
from io import BytesIO

# Convert DataFrame to CSV in memory
csv_buffer = BytesIO()
df_grouped.to_csv(csv_buffer, index=False)

# Move the buffer's position to the beginning
csv_buffer.seek(0)

def upload_csv_to_gcs(bucket_name, destination_blob_name, file_buffer):
    """Uploads a file from a buffer to Google Cloud Storage."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    # Upload the file-like object (buffer)
    blob.upload_from_file(file_buffer, content_type='text/csv')
    print(f"File uploaded to {destination_blob_name}.")

# Define your GCS bucket name and destination path
destination_blob_name = 'RS/processed-data-district/SRB_access_toschool_disctrict.csv'

# Upload the CSV from the buffer directly
upload_csv_to_gcs(bucket_name, destination_blob_name, csv_buffer)

File uploaded to RS/processed-data-district/SRB_access_toschool_disctrict.csv.
